In [1]:
import os

os.makedirs('../out/', exist_ok=True)

STATS_FILES = {
    'HSC_Lecco_2023': '../data/HSC_Lecco_2023-Players_Daily_Mobility_Stats.parquet',
    'HSC_Lecco_2024': '../data/HSC_Lecco_2024-Players_Daily_Mobility_Stats.parquet',
    'HSC_Ferrara_2023': '../data/HSC_Ferrara_2023-Players_Daily_Mobility_Stats.parquet',
}

SCORES_FILES = {
    'HSC_Lecco_2023': '../data/HSC_Lecco_2023-Players_Daily_Mobility_Scores.parquet',
    'HSC_Lecco_2024': '../data/HSC_Lecco_2024-Players_Daily_Mobility_Scores.parquet',
    'HSC_Ferrara_2023': '../data/HSC_Ferrara_2023-Players_Daily_Mobility_Scores.parquet',
}

In [2]:
import pandas as pd
import numpy as np

def load_stats(path: str, c: str) -> pd.DataFrame:
    df = pd.read_parquet(path)
    df['distance'] = df['distance'] / 1000 # convert to km
    df['campaign'] = c
    df.rename(columns={'modeType': 'counter', 'distance': 'score', 'stat_date': 'ts'}, inplace=True)
    return df[['playerId', 'campaign', 'ts', 'counter', 'score']]

def load_scores(path: str, c: str) -> pd.DataFrame:
    df = pd.read_parquet(path)
    df.rename(columns={'player_id': 'playerId', 'mobilityScore': 'score', 'day': 'ts'}, inplace=True)
    df['counter'] = 'score'
    df['campaign'] = c
    return df[['playerId', 'campaign', 'ts', 'counter', 'score']]

df_stats = pd.concat([load_stats(f, k) for k, f in STATS_FILES.items()])
df_scores = pd.concat([load_scores(f, k) for k, f in SCORES_FILES.items()])

df = pd.concat([df_stats, df_scores])
df['ts'] = pd.to_datetime(df['ts'])

df = df.groupby(['playerId', 'campaign', 'counter']).resample('W', on='ts').sum(numeric_only=True).reset_index()
df['score'] = df['score'].round(2)
df

,playerId,campaign,counter,ts,score
0,u_00144002f1614ee9a45f7822760e3746,HSC_Lecco_2023,score,2023-03-05,15.00
1,u_00144002f1614ee9a45f7822760e3746,HSC_Lecco_2023,walk,2023-03-05,1.21
2,u_00567a7bce8c4d09bea7db9bae375af4,HSC_Lecco_2023,bus,2023-03-05,6.48
3,u_00567a7bce8c4d09bea7db9bae375af4,HSC_Lecco_2023,bus,2023-03-12,0.00
4,u_00567a7bce8c4d09bea7db9bae375af4,HSC_Lecco_2023,bus,2023-03-19,0.00
...,...,...,...,...,...
20654,u_ffff190ed0ba42759b2f201c5e11ae0c,HSC_Lecco_2024,bus,2024-03-24,32.91
20655,u_ffff190ed0ba42759b2f201c5e11ae0c,HSC_Lecco_2024,score,2024-03-03,100.00
20656,u_ffff190ed0ba42759b2f201c5e11ae0c,HSC_Lecco_2024,score,2024-03-10,108.00
20657,u_ffff190ed0ba42759b2f201c5e11ae0c,HSC_Lecco_2024,score,2024-03-17,182.00


In [9]:
def flatten(df: pd.DataFrame):
    return pd.Series({
        'end': df['ts'].max(),
        'scores': df[['walk', 'bike', 'train', 'bus', 'car', 'score']].values
    })

flattened_df = df.pivot(index=['playerId', 'campaign', 'ts'], columns='counter', values='score').fillna(0).reset_index()
flattened_df = flattened_df.groupby(['playerId', 'campaign']).apply(flatten, include_groups=False).reset_index()
flattened_df = flattened_df[flattened_df['scores'].apply(lambda x: np.sum(x) > 0 and len(x) > 2)]
flattened_df.sort_values('end', inplace=True, ascending=False)
flattened_df

,playerId,campaign,end,scores
576,u_85cc0bcd70874e7a848d9c93a8f1dddc,HSC_Lecco_2024,2024-05-26,"[[7.35, 0.0, 0.0, 0.0, 0.0, 102.0], [29.28, 0...."
768,u_b3a0f0d2a6384fc8bae985f8aa66452a,HSC_Lecco_2024,2024-05-26,"[[0.24, 0.0, 0.0, 44.42, 0.0, 81.0], [0.0, 0.0..."
791,u_b93467ce38344365b56a10fbab8900fd,HSC_Lecco_2024,2024-05-26,"[[0.66, 0.0, 49.27, 0.0, 0.0, 54.0], [2.03, 0...."
212,u_33b55ca068884ac69db4aae7463d07ac,HSC_Lecco_2024,2024-05-26,"[[15.31, 0.0, 630.35, 0.0, 0.0, 660.0], [9.77,..."
788,u_b86600e67af94ed998279d5568e58941,HSC_Lecco_2024,2024-05-26,"[[10.67, 40.04, 0.0, 127.08, 0.0, 922.0], [3.5..."
...,...,...,...,...
720,u_a90ba34384ae4b009b7bc0af87b5b680,HSC_Lecco_2023,2023-03-19,"[[5.07, 0.0, 0.0, 0.31, 0.0, 73.0], [0.0, 0.0,..."
398,u_5a732d1fd4ed4b649fff0e728506324f,HSC_Lecco_2023,2023-03-19,"[[0.0, 0.0, 9.0, 0.0, 0.0, 9.0], [0.0, 0.0, 0...."
951,u_d92ecc4a21b946d7be868690d87d15c3,HSC_Lecco_2023,2023-03-12,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [6.79, 0.0, 5..."
668,u_9aa7ad833f8449849ee9ec3e62924591,HSC_Lecco_2023,2023-03-12,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [11.34, 0.0, ..."


In [10]:
flattened_df.groupby('end').size()

end
2023-03-12      3
2023-03-19     55
2023-03-26     39
2023-04-02     34
2023-04-09     37
2023-04-16     19
2023-04-23     26
2023-04-30     24
2023-05-07     21
2023-05-14     23
2023-05-21     40
2023-05-28    131
2023-06-04     42
2024-03-17     31
2024-03-24     40
2024-03-31     22
2024-04-07     23
2024-04-14     19
2024-04-21     14
2024-04-28      8
2024-05-05     14
2024-05-12     16
2024-05-19     16
2024-05-26    125
dtype: int64